In [22]:
%load_ext autoreload
%autoreload 2
%connect_info
%pprint 1
# General imports
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from typing import NamedTuple, List, Tuple, Set
from dataclasses import dataclass
from gt.solutions.br import BrownRobinsonOptimizer
from IPython.display import display

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')
logger = logging.getLogger(__name__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{
  "shell_port": 64525,
  "iopub_port": 64526,
  "stdin_port": 64527,
  "control_port": 64528,
  "hb_port": 64529,
  "ip": "127.0.0.1",
  "key": "cc3f7cd9-763cba8206e8961d3438bd17",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-98ea0d8c-bf5d-4907-b86c-1d1931cf517e.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
Pretty printing has been turned ON


In [4]:
def generate_random_game(limits: Tuple[int,int]=(0,100), 
                         shape: Tuple[int, int]=(10, 10)) -> Tuple[pd.DataFrame, pd.DataFrame]:
    if len(shape) != 2:
        raise NotImplementedError("Game matrix can be created only for two players!")
    def _random_matrix():
        return pd.DataFrame(data=np.random.randint(*limits, size=shape), 
                            index=[f'A{i+1}' for i in range(shape[0])], 
                            columns=[f'B{i+1}' for i in range(shape[0])])
    return _random_matrix(), _random_matrix()

def to_df(matrix: np.ndarray) -> pd.DataFrame:
    return pd.DataFrame(data=matrix,
                        index=[f'a{i+1}' for i in range(matrix.shape[0])],
                        columns=[f'b{i+1}' for i in range(matrix.shape[1])])

def cartesian(x, y):
    return np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])

In [9]:

#A, B = generate_random_game()
fugitives = (
    to_df(np.array([
        [-5, 0],
        [-10, -1],
    ])), 
    to_df(np.array([
        [-5, -10],
        [0, -1],
    ]))
)
variant = (
    to_df(np.array([
        [5, 10],
        [8, 6],
    ])), 
    to_df(np.array([
        [1, 4],
        [6, 9],
    ]))
)
random = generate_random_game(limits=(10, 99))
family = (
    to_df(np.array([
        [4, 0],
        [0, 1],
    ])), 
    to_df(np.array([
        [1, 0],
        [0, 4],
    ]))
)


Случайная игра 10x10:
Матрица игры:
(54 37) (42 31) (80 27) (35 24) (67 20) (48 22) (18 17) (57 70) (33 10) (15 54)
(38 21) (17 11) (39 89) (26 37) (60 33) (46 66) (26 83) (51 73) (33 86) (68 17)
(32 16) (64 36) (17 28) (84 84) (14 12) (48 72) (91 29) (68 93) (70 32) (66 40)
(80 60) (78 32) (85 48) (10 43) (47 57) (28 73) (35 70) (24 43) (56 44) (27 21)
(86 25) (75 73) (33 33) (87 36) (68 50) (85 41) (94 15) (47 16) (22 50) (35 37)
(58 73) (62 63) (94 29) (53 14) (27 33) (12 21) (29 76) (34 65) (25 45) (74 61)
(67 76) (61 74) (21 39) (49 59) (36 77) (76 66) (35 90) (76 35) (36 64) (27 65)
(97 30) (31 12) (37 32) (34 43) (28 26) (56 35) (89 87) (48 37) (56 26) (28 87)
(53 24) (88 39) (63 50) (34 86) (38 30) (55 98) (70 17) (40 55) (98 55) (59 64)
(64 12) (21 69) (29 27) (57 37) (92 66) (81 47) (80 55) (13 37) (64 12) (95 76)


In [10]:
def locate_pareto(A_matrix: pd.DataFrame, B_matrix: pd.DataFrame) -> Set[Tuple[str, str]]:
    if A_matrix.shape != B_matrix.shape:
        raise ValueError(f"Matrices for both players have different shapes: {A_matrix.shape} and {B_matrix.shape}")
    if not A_matrix.columns.equals(B_matrix.columns):
        raise ValueError(f"Matrices for both players have different columns: {A_matrix.columns} and {B_matrix.columns}")
    if not A_matrix.index.equals(B_matrix.index):
        raise ValueError(f"Matrices for both players have different rows: {A_matrix.index} and {B_matrix.index}")
    pareto_points = set()
    for pareto_column, pareto_row in cartesian(A_matrix.columns, A_matrix.index):
        logger.debug(f"[...] Looking at {pareto_column}, {pareto_row} "
                     f"({A_matrix[pareto_column][pareto_row]}, {B_matrix[pareto_column][pareto_row]})")
        for column, row in cartesian(A_matrix.columns, A_matrix.index):
            #logger.debug(f"- against {column}, {row} ({A_matrix[column][row]}, {B_matrix[column][row]})")
            if ((column != pareto_column or row != pareto_row)
                and A_matrix[pareto_column][pareto_row] < A_matrix[column][row]
                and B_matrix[pareto_column][pareto_row] < B_matrix[column][row]):
                logger.debug(f"[-] Pareto candidate {pareto_column}, {pareto_row} "
                             f"({A_matrix[pareto_column][pareto_row]}, {B_matrix[pareto_column][pareto_row]}) "
                             f"was dominated by {column}, {row} "
                             f"({A_matrix[column][row]}, {B_matrix[column][row]})")
                break
        else:
            logger.debug(f"[+] Adding point {pareto_column}, {pareto_row} "
                         f"({A_matrix[pareto_column][pareto_row]}, {B_matrix[pareto_column][pareto_row]}) "
                         f"as a real pareto point as no other point dominated this point.")
            pareto_points.add((pareto_column, pareto_row))
    return pareto_points

In [11]:
def locate_nash(A_matrix: pd.DataFrame, B_matrix: pd.DataFrame) -> Set[Tuple[str, str]]:
    if A_matrix.shape != B_matrix.shape:
        raise ValueError(f"Matrices for both players have different shapes: {A_matrix.shape} and {B_matrix.shape}")
    if not A_matrix.columns.equals(B_matrix.columns):
        raise ValueError(f"Matrices for both players have different columns: {A_matrix.columns} and {B_matrix.columns}")
    if not A_matrix.index.equals(B_matrix.index):
        raise ValueError(f"Matrices for both players have different rows: {A_matrix.index} and {B_matrix.index}")
    
    logger.info(f"Maxima for player A (column): \n{A_matrix.max(axis=0)}")
    br = A_matrix.idxmax(axis=0)
    player_a_responses = {(i, br[i]) for i in br.index}
    logger.info(f"Best responses for A: {player_a_responses}")

    logger.info(f"Maxima for player B (row): \n{B_matrix.max(axis=1)}")
    br = B_matrix.idxmax(axis=1)
    player_b_responses = {(br[i], i) for i in br.index}
    logger.info(f"Best responses for B: {player_b_responses}")
    return player_b_responses.intersection(player_a_responses)



In [20]:
def report(game_name: str, A_matrix, B_matrix, title_padding:int=40):
    nash = locate_nash(A_matrix, B_matrix)
    pareto = locate_pareto(A_matrix, B_matrix)
    print("=" * title_padding)
    game_pad = max(0, (title_padding - len(game_name))//2 - 1)
    print("=" * game_pad + f" {game_name} " + "=" * game_pad)
    print(f"{game_name} {random[0].shape[0]}x{random[0].shape[1]}:")
    printed_matrix = "\n".join([
        "  ".join([f"({A_matrix.iloc[i].iloc[j]} {B_matrix.iloc[i].iloc[j]})" 
                  for j in range(A_matrix.shape[0])])
        for i in range(A_matrix.shape[1])
    ])
    print(f"Матрица игры:\n{printed_matrix}")
    print("=" * title_padding)
    print("REWARDS FOR A")
    display(A_matrix)
    print("REWARDS FOR B")
    display(B_matrix)
    print(f"НЭШ: {nash}")
    print(f"ОПТИМУМ ПАРЕТО: {pareto}")
    print(f"ПЕРЕСЕЧЕНИЕ: {pareto.intersection(nash)}")
    print("=" * title_padding)

In [21]:
report("Дилема заключённых", *fugitives)
report("9 Вариант", *variant)
report("Случайная игра", *random)
report("Семейный спор", *family)

12:08:58 INFO:Maxima for player A (column): 
b1   -5
b2    0
dtype: int32
12:08:58 INFO:Best responses for A: {('b1', 'a1'), ('b2', 'a1')}
12:08:58 INFO:Maxima for player B (row): 
a1   -5
a2    0
dtype: int32
12:08:58 INFO:Best responses for B: {('b1', 'a1'), ('b1', 'a2')}
12:08:58 DEBUG:[...] Looking at b1, a1 (-5, -5)
12:08:58 DEBUG:[-] Pareto candidate b1, a1 (-5, -5) was dominated by b2, a2 (-1, -1)
12:08:58 DEBUG:[...] Looking at b2, a1 (0, -10)
12:08:58 DEBUG:[+] Adding point b2, a1 (0, -10) as a real pareto point as no other point dominated this point.
12:08:58 DEBUG:[...] Looking at b1, a2 (-10, 0)
12:08:58 DEBUG:[+] Adding point b1, a2 (-10, 0) as a real pareto point as no other point dominated this point.
12:08:58 DEBUG:[...] Looking at b2, a2 (-1, -1)
12:08:58 DEBUG:[+] Adding point b2, a2 (-1, -1) as a real pareto point as no other point dominated this point.


========== Дилема заключённых ==========
Дилема заключённых 10x10:
Матрица игры:
(-5 -5)  (0 -10)
(-10 0)  (-1 -1)
REWARDS FOR A


,b1,b2
a1,-5,0
a2,-10,-1


REWARDS FOR B


,b1,b2
a1,-5,-10
a2,0,-1


...

НЭШ: {('B10', 'A10')}
ОПТИМУМ ПАРЕТО: {('B10', 'A10'), ('B7', 'A8'), ('B8', 'A3'), ('B9', 'A9'), ('B6', 'A9')}
ПЕРЕСЕЧЕНИЕ: {('B10', 'A10')}
============ Семейный спор ============
Семейный спор 10x10:
Матрица игры:
(4 1)  (0 0)
(0 0)  (1 4)
REWARDS FOR A


,b1,b2
a1,4,0
a2,0,1


REWARDS FOR B


,b1,b2
a1,1,0
a2,0,4


НЭШ: {('b1', 'a1'), ('b2', 'a2')}
ОПТИМУМ ПАРЕТО: {('b1', 'a1'), ('b2', 'a2')}
ПЕРЕСЕЧЕНИЕ: {('b1', 'a1'), ('b2', 'a2')}
